In [60]:
import sys
sys.path.append('/home/antoine/Documents/GitHub/Link-Encyclo')
sys.path.append('/home/antoine/Documents/GitHub/datas')
from EncycloObject import Article, Book
from WikidataTools import WikiDataObject as wdoooo
import os
import folium
import pickle
import requests

In [56]:
with open('sample_france.pkl', 'rb') as f:
    sample_france = pickle.load(f)

**Input**
- `sample` : a `Book` instance of articles, each having the attribute `preds`
- `sample_name` : a string such as `champagne_france`: the name of the sample we are working on
- `id_preds` the Key to look or inside the predictions, so we can reach them with `article.preds['id_preds']`
- optional: `bbox` : the bounding box to plot
```python
folium.Rectangle(
bounds=[[1, 2], [8, 9]],
color='green',
fill=False,
)
```

Structure de preds :
- Liste de dict 
- avec comme keys : 'rank' (1,2, 3...), 'pred_id' (qids), 'pred_coords'(tuple), 'acc@10' (booléen)



**Output** 
- html maps are saved under `output/maps_{id_preds}/sample_name`
- `final_map`: a Folium map

**problèmes**
- si je veux plotter (champagne, france) ET (suabe, allemagne) --> 2 commus

In [61]:
def retrieve_infos(qid,
                   tupling = True,
                   solr_instance = 'http://localhost:8983/solr/frenchtapioca5/query'):
    """Retrieve the coordinates of a qid
    Given the Solr collection available @ solr_instance"""
    params = {
        'q': 'id:{}'.format(qid),
        'wt': 'json',
        'fl': 'label,coordinates,desc'
    }
    try:
        r = requests.post(solr_instance, data=params)
        r.raise_for_status()
        response_json = r.json()
        if len(response_json['response']['docs']) == 0:
            return None, None, None
        coords = response_json['response']['docs'][0].get('coordinates')
        label = response_json['response']['docs'][0].get('label')
        desc = response_json['response']['docs'][0].get('desc')
        if tupling:
            return tuple(map(float, coords[0].split(','))), label, desc
        else:
            return coords, label, desc
    except (requests.RequestException, KeyError, IndexError) as e:
        return None

retrieve_infos(qid = 'Q142', tupling=True)

((47.0, 2.0),
 ['France'],
 "pays transcontinental au territoire métropolitain en Europe de l'Ouest")

In [62]:
def preds2map(sample, sample_name, preds_id, bbox=None):
    """mapping"""
    
    m = folium.Map(location=[48.8566, 2.3522], zoom_start=5)
    
    for article in sample:
        
        # are there any predictions ?
        predictions = article.preds[preds_id]
        if not predictions:
            continue

        # get the best prediction
        best_candidate = predictions.pop(0)
        if best_candidate['pred_id'] == 'Q0': # Entity linking failed
            continue
        ressource = wdo(best_candidate.get('pred_id'))
        label = ressource._get_label(lang='en')
        desc = ressource._get_description(lang='en')

        # we have gold coordinates 
        if hasattr(article, 'gold_nugues'):
            gap = best_candidate['acc@10']
            gold = wdo(article.gold_nugues)
            gold_label = gold._get_label(lang='en')
            gold_desc = gold._get_description(lang='en')
            marker_color = 'green' if gap else 'red'
            html = f"""
            <html>
                <body>
                    <a href="{article.artfl}" target="_blank"><strong>{article.hash}</strong></a>
                    <p style="color: #555;">{article.text[:150]}...</p>
                    <p><strong>GOLD :</strong> <a href="{gold.link}" target="_blank">{gold_label}</a>, {gold_desc}, {gold.uri}</p>
                    <p><strong>PRED n°{best_candidate['rank']} : <a href="{ressource.link}" target="_blank">{label}</a>, {desc}, {ressource.uri}</p>
                </body>
            </html>
            """
            # delete wdo objects
            del gold
            del ressource

        # we don't have gold coordinates <p><strong> Best prediction :</strong> <a href="{wdo(best_candidate['pred_id']).link}" target="_blank">{best_candidate['pred_id']}</a></p>
        else:    
            marker_color = 'beige'            
            html = f"""
            <!DOCTYPE html>
            <html>
            <head>
            <title>Marker Popup</title>
            </head>
            <body>
            <a href="{article.artfl}" target="_blank"><strong>{article.hash}</strong></a>
            <p style="color: #555;">{article.text[:150]}...</p>
            <p> <strong>PRED n°{best_candidate['rank']} : <a href="{ressource.link}" target="_blank">{label}</a>, {desc}, {ressource.uri}</p>
            <form action="http://localhost:5000/save_annotation" method="post" target="hidden_iframe">
                <label><input type="checkbox" name="correct" value="correct"> Correct</label><br>
                <label><input type="checkbox" name="wrong" value="wrong"> Wrong</label><br>
                <label><input type="checkbox" name="abberant" value="abberant"> Abberant</label><br>
                <label for="annotation">Annotation suggérée : </label><br>
                <textarea id="annotation" name="annotation" rows="1" cols="30"></textarea><br>
                <input type="hidden" id="hash" name="hash" value="{article.hash}">
                <label for="note">Remarque : </label><br>
                <textarea id="note" name="note" rows="1" cols="30"></textarea><br>
                <input type="submit" value="Sauvegarder">
            <p><strong> Other predictions :</strong> </p>
            """

            for pred in predictions :
                ressource = wdo(pred['pred_id'])
                label = ressource._get_label(lang='en')
                desc = ressource._get_description(lang='en')
                html += f"""
                <p>n°{pred['rank']} : <a href="{ressource.link}" target="_blank">{label}</a>, {desc}, {ressource.uri}</p>
                """

            html += """
            </form>
            <iframe name="hidden_iframe" style="display:none;"></iframe>
            </body>
            </html>
            """

            del ressource

        # Add the marker to the map
        folium.Marker(
            location=best_candidate['pred_coords'],
            popup=folium.Popup(html, max_width=500),
            icon=folium.Icon(color=marker_color)
        ).add_to(m)

        # add the bbox to the map
        if bbox:
            bbox.add_to(m)

    # save the map and return it
    # if no foler output/maps_{preds_id} exists, create it
    if not os.path.exists(f'outputs/maps_{preds_id}'):
        os.makedirs(f'outputs/maps_{preds_id}')
    m.save(f'outputs/maps_{preds_id}/{sample_name}.html')
    return m


In [58]:
m = preds2map(sample = sample_france[500:550], sample_name = 'test_france', preds_id = 'test_france')

In [64]:
import json
print(json.dumps(sample_france[0].preds, indent=2))

{
  "test_france": [
    {
      "rank": 0,
      "pred_id": "Q3390498",
      "pred_coords": [
        48.8458,
        2.37243
      ],
      "acc@10": false,
      "label": [
        "place des Combattants-en-Afrique-du-Nord"
      ],
      "desc": "place de Paris, France"
    },
    {
      "rank": 1,
      "pred_id": "Q106643503",
      "pred_coords": [
        48.880970675,
        2.42174555
      ],
      "acc@10": false,
      "label": [
        "avenue des Combattants-d'Afrique-du-Nord"
      ],
      "desc": "avenue des Lilas, en France"
    },
    {
      "rank": 2,
      "pred_id": "Q110955946",
      "pred_coords": [
        43.28810336792453,
        5.412446551886793
      ],
      "acc@10": false,
      "label": [
        "chemin de l'Arm\u00e9e d'Afrique"
      ],
      "desc": "chemin de Marseille, en France"
    },
    {
      "rank": 3,
      "pred_id": "Q113448661",
      "pred_coords": [
        48.964191957142866,
        2.524119314285714
      ],
      "acc@10